# Trabalho Final - INF 493

> Alunos: Cleber Luiz Oliveira Junior, Vinicius Kuster Lodi


#IMPORTS IMPORTANTES

In [ ]:
# Imports iniciais:

import pandas as pd
import matplotlib as mtplt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [ ]:
#!pip install scikit-optimize catboost xgboost

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Treinamento dos modelos

## Seleção de modelos

### Divisão em treino e teste com k=10 e n=30 para validação cruzada/estratificada

In [ ]:
base = pd.read_excel("basePosProcessamento2500Normalizada.xlsx")

In [ ]:
base.describe().T

,count,mean,std,min,25%,50%,75%,max
UF_NOTIF,2486.0,-0.118396,1.920501,-7.666667,-1.000000,0.000000,0.000000,6.333333
IDADE,2486.0,0.005257,0.659626,-1.344828,-0.517241,-0.034483,0.482759,1.931034
SEXO,2486.0,-0.317377,0.465549,-1.000000,-1.000000,0.000000,0.000000,0.000000
VACINA_FA,2486.0,-0.382542,0.611494,-1.000000,-1.000000,0.000000,0.000000,1.000000
SINT_HEMORRAGICO,2486.0,-0.030169,0.574697,-1.000000,0.000000,0.000000,0.000000,1.000000
DISTUR_RENAL,2486.0,-0.044248,0.596142,-1.000000,0.000000,0.000000,0.000000,1.000000
SINAL_FAGET,2486.0,0.052695,0.545734,-1.000000,0.000000,0.000000,0.000000,1.000000
DOR_ABDOMINAL,2486.0,-0.034191,0.420833,-0.500000,-0.500000,0.000000,0.500000,0.500000
RESULT_IGM,2486.0,0.114642,0.634465,-1.000000,0.000000,0.000000,1.000000,1.000000
RESULT_PCR,2486.0,0.202333,0.700453,-1.000000,0.000000,0.000000,1.000000,1.000000


In [ ]:
semente = 42

y = base['OBITO']
X = base.drop(columns=['OBITO'])

print("Shape de X:", X.shape)
print("Shape de y:", y.shape)


Shape de X: (2486, 17)
Shape de y: (2486,)


### Instanciação dos modelos padrão

In [ ]:
modelos = {
    'Reg. Log.': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(
            random_state=semente,
            n_jobs=-1,
            max_iter=1000
        ))
    ]),
    'Árv. Dec.': DecisionTreeClassifier(random_state=semente),
    'K Viz.': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier(
            n_neighbors=10,
            n_jobs=-1
        ))
    ]),
    'Bayes': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', GaussianNB())
    ]),
    'Rand. For.': RandomForestClassifier(
        random_state=semente,
        n_jobs=-1
    ),

    'AdaBoost': AdaBoostClassifier(
        random_state=semente
    ),
    'CatBoost': CatBoostClassifier(
        random_state=semente,
        verbose=0,
        thread_count=-1
    ),
    'XGBoost': XGBClassifier(
        random_state=semente,
        n_jobs=-1,
        eval_metric='logloss'
    ),
}

### Avaliação inicial dos modelos

In [ ]:
rskf_avaliacao = RepeatedStratifiedKFold(
    n_splits=10,
    n_repeats=30,
    random_state=semente
)

results = {}

print("\n==============================")
print("AVALIAÇÃO INICIAL DOS MODELOS")
print("================================")

for name, base_model in modelos.items():
    print(f"\nAvaliando {name}...")
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []

    for train_index, test_index in rskf_avaliacao.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = clone(base_model)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_proba = None

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, zero_division=0))
        roc_auc_scores.append(
            roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
        )

    results[name] = {
        'Accuracy': {'mean': np.mean(accuracy_scores), 'std': np.std(accuracy_scores)},
        'Precision': {'mean': np.mean(precision_scores), 'std': np.std(precision_scores)},
        'Recall': {'mean': np.mean(recall_scores), 'std': np.std(recall_scores)},
        'F1-Score': {'mean': np.mean(f1_scores), 'std': np.std(f1_scores)},
        'ROC AUC': {'mean': np.nanmean(roc_auc_scores), 'std': np.nanstd(roc_auc_scores)}
    }

# Imprime os resultados
print("\n\nRESULTADOS (MÉDIA ± DESVIO PADRÃO):")
for name, metrics in results.items():
    print(f"\n--- {name} ---")
    for metric_name, values in metrics.items():
        print(f"{metric_name}: {values['mean']:.4f} ± {values['std']:.4f}")


AVALIAÇÃO INICIAL DOS MODELOS

Avaliando Reg. Log....

Avaliando Árv. Dec....

Avaliando K Viz....

Avaliando Bayes...

Avaliando Rand. For....

Avaliando AdaBoost...

Avaliando CatBoost...

Avaliando XGBoost...


RESULTADOS (MÉDIA ± DESVIO PADRÃO):

--- Reg. Log. ---
Accuracy: 0.8104 ± 0.0221
Precision: 0.8066 ± 0.0375
Recall: 0.6885 ± 0.0445
F1-Score: 0.7419 ± 0.0323
ROC AUC: 0.8542 ± 0.0241

--- Árv. Dec. ---
Accuracy: 0.8423 ± 0.0216
Precision: 0.7985 ± 0.0345
Recall: 0.8079 ± 0.0379
F1-Score: 0.8025 ± 0.0268
ROC AUC: 0.8364 ± 0.0223

--- K Viz. ---
Accuracy: 0.8483 ± 0.0208
Precision: 0.9494 ± 0.0261
Recall: 0.6525 ± 0.0475
F1-Score: 0.7725 ± 0.0366
ROC AUC: 0.9138 ± 0.0188

--- Bayes ---
Accuracy: 0.7876 ± 0.0591
Precision: 0.9445 ± 0.0731
Recall: 0.4840 ± 0.1490
F1-Score: 0.6264 ± 0.1763
ROC AUC: 0.9094 ± 0.0185

--- Rand. For. ---
Accuracy: 0.9014 ± 0.0175
Precision: 0.9271 ± 0.0282
Recall: 0.8161 ± 0.0356
F1-Score: 0.8675 ± 0.0247
ROC AUC: 0.9446 ± 0.0151

--- AdaBoost ---
Ac

### Busca bayesiana de hiperparametros

In [ ]:
cv_bayes = RepeatedStratifiedKFold(
    n_splits=5,      # 5 folds
    n_repeats=3,     # repetidos 3x -> 15 avaliações por combinação
    random_state=semente
)

# Espaços de busca bayesiana para cada modelo
search_spaces = {
    'Reg. Log.': {
        'scaler': Categorical([StandardScaler(), 'passthrough']),
        'clf__C': Real(1e-4, 1e3, prior='log-uniform'),
        'clf__penalty': Categorical(['l1', 'l2']),
        'clf__solver': Categorical(['liblinear', 'saga']),
        'clf__class_weight': Categorical([None, 'balanced']),
        'clf__max_iter': Integer(200, 1000),
    },

    'Árv. Dec.': {
        'criterion': Categorical(['gini', 'entropy']),
        'max_depth': Categorical([None, 5, 10, 20, 30]),
        'min_samples_split': Integer(2, 20),
        'min_samples_leaf': Integer(1, 10),
        'max_features': Categorical([None, 'sqrt', 'log2']),
        'class_weight': Categorical([None, 'balanced']),
    },

    'K Viz.': {
        'scaler': Categorical([StandardScaler(), 'passthrough']),
        'clf__n_neighbors': Integer(1, 25),
        'clf__leaf_size': Integer(10, 50),
        'clf__weights': Categorical(['uniform', 'distance']),
        'clf__p': Integer(1, 2),  # 1 = manhattan, 2 = euclidiana
    },

    'Rand. For.': {
        'n_estimators': Integer(100, 800),
        'max_depth': Categorical([None, 5, 10, 20, 30]),
        'min_samples_split': Integer(2, 20),
        'min_samples_leaf': Integer(1, 10),
        'max_features': Categorical(['sqrt', 'log2', None]),
        'bootstrap': Categorical([True, False]),
        'class_weight': Categorical([None, 'balanced']),
    },

    'AdaBoost': {
        'n_estimators': Integer(50, 300),
        'learning_rate': Real(1e-3, 1.0, prior='log-uniform'),
        'estimator': Categorical([
            DecisionTreeClassifier(max_depth=1),
            DecisionTreeClassifier(max_depth=2),
            DecisionTreeClassifier(max_depth=3),
        ])
    },

    'CatBoost': {
        'iterations': Integer(50, 500),
        'learning_rate': Real(1e-3, 0.3, prior='log-uniform'),
        'depth': Integer(3, 10),
        'l2_leaf_reg': Real(1, 10, prior='log-uniform'),
        'bagging_temperature': Real(0.0, 1.0),
        'border_count': Integer(32, 254),
        'scale_pos_weight': Real(1.0, 10.0),
    },

    'XGBoost': {
        'n_estimators': Integer(100, 500),
        'learning_rate': Real(1e-3, 0.3, prior='log-uniform'),
        'max_depth': Integer(3, 10),
        'subsample': Real(0.6, 1.0),
        'colsample_bytree': Real(0.6, 1.0),
        'min_child_weight': Integer(1, 10),
        'gamma': Real(0.0, 0.3),
        'reg_lambda': Real(1.0, 10.0, prior='log-uniform'),
        'reg_alpha': Real(1e-8, 1e-1, prior='log-uniform'),
    },
}

tuned_results = {}
best_estimators = {}

### Execução da busca bayesiana

In [ ]:
# número de iterações de busca bayesiana (por modelo)
N_ITER = 50

for name, model in modelos.items():
    if name not in search_spaces:
        print(f"Nenhum espaço de busca definido para {name}. Pulando.\n")
        continue

    print(f"-> Iniciando busca bayesiana para {name}...")

    opt = BayesSearchCV(
        estimator=model,
        search_spaces=search_spaces[name],
        n_iter=N_ITER,
        scoring='roc_auc',
        cv=cv_bayes,
        n_jobs=-1,
        random_state=semente,
        verbose=2,
        refit=True  # re-treina no conjunto completo com os melhores hiperparâmetros
    )

    opt.fit(X, y)

    tuned_results[name] = {
        'Best ROC AUC': opt.best_score_,
        'Best Parameters': opt.best_params_
    }
    best_estimators[name] = opt.best_estimator_

    print(f"  {name} concluído. Melhor ROC AUC: {opt.best_score_:.4f}")
    print(f"  Melhores parâmetros: {opt.best_params_}\n")

print("\n--- RESUMO DO AJUSTE FINO (BUSCA BAYESIANA) ---")
for name, metrics in tuned_results.items():
    print(f"Modelo: {name}")
    print(f"  Melhor ROC AUC: {metrics['Best ROC AUC']:.4f}")
    print(f"  Melhores Parâmetros: {metrics['Best Parameters']}")
    print("-" * 50)

-> Iniciando busca bayesiana para Reg. Log....
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candida

C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


  Reg. Log. concluído. Melhor ROC AUC: 0.8536
  Melhores parâmetros: OrderedDict({'clf__C': 23.688572048728528, 'clf__class_weight': 'balanced', 'clf__max_iter': 200, 'clf__penalty': 'l1', 'clf__solver': 'liblinear', 'scaler': StandardScaler()})

-> Iniciando busca bayesiana para Árv. Dec....
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 cand

C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('balanced'), np.str_('entropy'), np.int64(10), None, np.int64(10), np.int64(20)] before, using random point ['balanced', 'gini', 30, 'sqrt', np.int64(3), np.int64(10)]
  warnings.warn(


Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits


C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('balanced'), np.str_('entropy'), np.int64(10), np.str_('log2'), np.int64(6), np.int64(20)] before, using random point ['balanced', 'entropy', 10, 'log2', np.int64(8), np.int64(8)]
  warnings.warn(


Fitting 15 folds for each of 1 candidates, totalling 15 fits


C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [np.str_('balanced'), np.str_('entropy'), np.int64(10), np.str_('log2'), np.int64(6), np.int64(20)] before, using random point ['balanced', 'gini', 20, 'log2', np.int64(2), np.int64(11)]
  warnings.warn(


Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
  Árv. Dec. concluído. Melhor ROC AUC: 0.9093
  Melhores parâmetros: OrderedDict({'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 19})

-> Iniciando busca bayesiana para K Viz....
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 1

### Busca bayesiana refinada nos TOP-K modelos

In [ ]:
TOP_K = 2

# Ordena modelos pelo melhor ROC AUC da primeira rodada de tuning
sorted_models = sorted(
    tuned_results.items(),
    key=lambda x: x[1]['Best ROC AUC'],
    reverse=True
)

top_models_names = [name for name, _ in sorted_models[:TOP_K]]

print("\n===============================================")
print("ETAPA 2: REFINO BAYESIANO DOS TOP MODELOS")
print("Top modelos selecionados:", top_models_names)
print("===============================================\n")

# CV mais forte
cv_bayes_fino = RepeatedStratifiedKFold(
    n_splits=10,
    n_repeats=5,
    random_state=semente
)

N_ITER_FINE = 60  # mais iterações de busca bayesiana no refino

tuned_results_fino = {}
best_estimators_fino = {}

for name in top_models_names:
    print(f"-> Iniciando busca bayesiana REFINADA para {name}...")

    if name not in search_spaces:
        print(f"  Nenhum espaço de busca definido para {name}. Pulando.\n")
        continue

    base_estimator = modelos[name]
    space = search_spaces[name]

    opt_fino = BayesSearchCV(
        estimator=base_estimator,
        search_spaces=space,
        n_iter=N_ITER_FINE,
        scoring='roc_auc',
        cv=cv_bayes_fino,
        n_jobs=-1,
        random_state=semente,
        verbose=2,
        refit=True
    )

    opt_fino.fit(X, y)

    tuned_results_fino[name] = {
        'Best ROC AUC': opt_fino.best_score_,
        'Best Parameters': opt_fino.best_params_
    }
    best_estimators_fino[name] = opt_fino.best_estimator_

    print(f"  Refino de {name} concluído.")
    print(f"  Melhor ROC AUC (refino): {opt_fino.best_score_:.4f}")
    print(f"  Melhores parâmetros (refino): {opt_fino.best_params_}\n")

print("\n--- RESUMO DO REFINO BAYESIANO (TOP-K) ---")
for name, metrics in tuned_results_fino.items():
    print(f"Modelo: {name}")
    print(f"  Melhor ROC AUC (refino): {metrics['Best ROC AUC']:.4f}")
    print(f"  Melhores Parâmetros (refino): {metrics['Best Parameters']}")
    print("-" * 50)


ETAPA 2: REFINO BAYESIANO DOS TOP MODELOS
Top modelos selecionados: ['CatBoost', 'Rand. For.']

-> Iniciando busca bayesiana REFINADA para CatBoost...
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50 fits
Fitting 50 folds for each of 1 candidates, totalling 50

### Avaliação final (10×30) com os melhores modelos

In [ ]:
print("\n===============================================")
print("ETAPA 3: AVALIAÇÃO FINAL (10x30) DOS MODELOS")
print("===============================================\n")

# Escolhe, para cada modelo, qual versão será usada na avaliação final:
# 1) Se tiver versão refinada (tuned_results_fino), usa essa
# 2) Senão, usa a versão da primeira busca bayesiana (best_estimators)
# 3) Se nem isso existir, usa o modelo base

final_models = {}

for name in modelos.keys():
    if name in best_estimators_fino:
        final_models[name] = best_estimators_fino[name]
    elif name in best_estimators:
        final_models[name] = best_estimators[name]
    else:
        final_models[name] = modelos[name]

# CV para avaliação final
rskf_final = RepeatedStratifiedKFold(
    n_splits=10,
    n_repeats=30,
    random_state=semente
)

final_results = {}

for name, final_model in final_models.items():
    print(f"Avaliando modelo FINAL: {name} ...")

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []

    for train_idx, test_idx in rskf_final.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(final_model)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Probabilidades (se disponíveis) para ROC AUC
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_proba = None

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, zero_division=0))
        roc_auc_scores.append(
            roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
        )

    final_results[name] = {
        'Accuracy': {
            'mean': np.mean(accuracy_scores),
            'std': np.std(accuracy_scores)
        },
        'Precision': {
            'mean': np.mean(precision_scores),
            'std': np.std(precision_scores)
        },
        'Recall': {
            'mean': np.mean(recall_scores),
            'std': np.std(recall_scores)
        },
        'F1-Score': {
            'mean': np.mean(f1_scores),
            'std': np.std(f1_scores)
        },
        'ROC AUC': {
            'mean': np.nanmean(roc_auc_scores),
            'std': np.nanstd(roc_auc_scores)
        }
    }

print("\n===============================================")
print("RESULTADOS FINAIS – MÉDIA ± DESVIO (CV 10x30)")
print("=================================================")

for name, metrics in final_results.items():
    print(f"\n--- {name} ---")
    for metric_name, values in metrics.items():
        print(f"{metric_name}: {values['mean']:.4f} ± {values['std']:.4f}")

# Ranking por ROC AUC final
ranking_auc = sorted(
    final_results.items(),
    key=lambda x: x[1]['ROC AUC']['mean'],
    reverse=True
)

print("\n===============================================")
print("RANKING POR ROC AUC (AVALIAÇÃO FINAL 10x30)")
print("=================================================")
for pos, (name, metrics) in enumerate(ranking_auc, start=1):
    print(f"{pos}. {name}: ROC AUC = {metrics['ROC AUC']['mean']:.4f} ± {metrics['ROC AUC']['std']:.4f}")


ETAPA 3: AVALIAÇÃO FINAL (10x30) DOS MODELOS

Avaliando modelo FINAL: Reg. Log. ...


C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
C:\Users\vinic\PyCharmMiscProject\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jo

Avaliando modelo FINAL: Árv. Dec. ...
Avaliando modelo FINAL: K Viz. ...
Avaliando modelo FINAL: Bayes ...
Avaliando modelo FINAL: Rand. For. ...
Avaliando modelo FINAL: AdaBoost ...
Avaliando modelo FINAL: CatBoost ...
Avaliando modelo FINAL: XGBoost ...

RESULTADOS FINAIS – MÉDIA ± DESVIO (CV 10x30)

--- Reg. Log. ---
Accuracy: 0.7918 ± 0.0235
Precision: 0.7298 ± 0.0345
Recall: 0.7574 ± 0.0425
F1-Score: 0.7425 ± 0.0292
ROC AUC: 0.8551 ± 0.0239

--- Árv. Dec. ---
Accuracy: 0.8618 ± 0.0208
Precision: 0.8652 ± 0.0363
Recall: 0.7736 ± 0.0412
F1-Score: 0.8160 ± 0.0289
ROC AUC: 0.9097 ± 0.0186

--- K Viz. ---
Accuracy: 0.8585 ± 0.0195
Precision: 0.9455 ± 0.0260
Recall: 0.6828 ± 0.0443
F1-Score: 0.7922 ± 0.0328
ROC AUC: 0.9277 ± 0.0166

--- Bayes ---
Accuracy: 0.7876 ± 0.0591
Precision: 0.9445 ± 0.0731
Recall: 0.4840 ± 0.1490
F1-Score: 0.6264 ± 0.1763
ROC AUC: 0.9094 ± 0.0185

--- Rand. For. ---
Accuracy: 0.9016 ± 0.0168
Precision: 0.9265 ± 0.0267
Recall: 0.8173 ± 0.0361
F1-Score: 0.8679 ± 